In [ ]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 89kB/s 


In [ ]:
import os

import numpy as np
import pandas as pd

#import pydicom
import cv2
import matplotlib.pyplot as plt

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.utils import plot_model
from google.colab.patches import cv2_imshow

from glob import glob
from tqdm import tqdm

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
ini_path = "/content/drive/My Drive/";
INPUT_DIR = os.path.join(ini_path)

SEGMENTATION_DIR = os.path.join(INPUT_DIR, "final_report_images")
SEGMENTATION_MODEL = os.path.join(ini_path, "fin_seg_mod.hdf5")
SEGMENTATION_RESULT_TEST = os.path.join(SEGMENTATION_DIR)

RSNA_DIR = os.path.join(INPUT_DIR)
RSNA_TEST_DIR = os.path.join(RSNA_DIR, "final_report_images","source")
print(SEGMENTATION_RESULT_TEST)
print(RSNA_TEST_DIR)

NameError: ignored

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

segmentation_model = load_model(SEGMENTATION_MODEL, \
                                custom_objects={'dice_coef_loss': dice_coef_loss, \
                                                'dice_coef': dice_coef})

segmentation_model.summary()

In [ ]:
plot_model(segmentation_model)

In [ ]:
def image_to_train(img):
    npy = img / 255
    npy = np.reshape(npy, npy.shape + (1,))
    npy = np.reshape(npy,(1,) + npy.shape)
    npy.shape
    return npy

def train_to_image(npy):
    img = (npy[0,:, :, 0] * 255.).astype(np.uint8)
    return img

In [ ]:
def gammaCorrection(img_original, gamma):
    ## [changing-contrast-brightness-gamma-correction]
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)

    res = cv2.LUT(img_original, lookUpTable)
    ## [changing-contrast-brightness-gamma-correction]

    #img_gamma_corrected = cv2.hconcat([img_original, res])
    return res

def adjust_gamma(image, gamma=1.0):
   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

   return cv2.LUT(image, table)
name = "post-intubuation-pneumomediastium-and-pneumothorax-background-covid-19-pneumonia-day6-1.jpg"
img = cv2.imread(os.path.join(RSNA_TEST_DIR,name))
#cv2_imshow(img)
img2 = adjust_gamma(img,gamma=2)
img3 = gammaCorrection(img,0.3325)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,32))
img3 = clahe.apply(cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY))
#cv2_imshow(np.hstack((img,img2)))
#cv2_imshow(np.hstack((img2,img3)))
cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, ('1'+name)), img)
cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, ('2'+name)), img2)
cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, ('320'+name)), img3)

True

# **Test on single file**

In [ ]:
def gammaCorrection(img_original, gamma):
    ## [changing-contrast-brightness-gamma-correction]
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)

    res = cv2.LUT(img_original, lookUpTable)
    ## [changing-contrast-brightness-gamma-correction]

    #img_gamma_corrected = cv2.hconcat([img_original, res])
    return res

def adjust_gamma(image, gamma=1.0):
   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

   return cv2.LUT(image, table)

def segment_image(pid, img, save_to):
    img = cv2.resize(img, (512, 512))
    segm_ret = segmentation_model.predict(image_to_train(img), \
                                          verbose=0)

    img = cv2.bitwise_and(img, img, mask=train_to_image(segm_ret))
    pid += '3'
    #cv2.imwrite(os.path.join(save_to, "%s.png" % pid), img)
    return img

name = "post-intubuation-pneumomediastium-and-pneumothorax-background-covid-19-pneumonia-day6-1.jpg"

for i in range(1,4):
  name2 = str(i)+name
  pid, fileext = os.path.splitext(name2)
  filename = os.path.join(SEGMENTATION_RESULT_TEST,name2)
  img = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
  width, height, nChannels, nImages = (512, 512, 0, 2)
  images1 = np.zeros((width, height))
  images2 = np.zeros((width, height))
  images3 = np.zeros((width, height))
  images4 = np.zeros((width, height))
  images5 = np.zeros((width, height))
  img = cv2.equalizeHist(img)
  clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,32))
  img2 = clahe.apply(img)
  #clahe = cv2.createCLAHE(clipLimit=0.8, tileGridSize=(8,32))
  #img3 = adjust_gamma(img,gamma=1.5)
  img3 = gammaCorrection(img,0.3325)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,32))
  clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8,32))
  img4 = clahe.apply(img)
  img = segment_image(pid, img, SEGMENTATION_RESULT_TEST)
  img2 = segment_image(pid, img2, SEGMENTATION_RESULT_TEST)
  img3 = segment_image(pid, img3, SEGMENTATION_RESULT_TEST)
  img4 = segment_image(pid, img4, SEGMENTATION_RESULT_TEST)
  images1[:, :] = cv2.resize(img, (width, height))
  images2[:, :] = cv2.resize(img2, (width, height))
  images3[:, :] = cv2.resize(img3, (width, height))
  images4[:, :] = cv2.resize(img4, (width, height))
  for i in range(0,width):
    for j in range(0,height):
      if (images1[i,j]!=0 and images2[i,j]!=0 and images3[i,j]!=0 and images4[i,j]!=0):
        images5[i,j] = images2[i,j]
      elif (images2[i,j]==0):
        if (images1[i,j]==0):
          images5[i,j] = images3[i,j]
        else:
          images5[i,j] = images1[i,j]
      else:
        images5[i,j] = images2[i,j]
  #cv2_imshow(img)
  #cv2_imshow(img2)
  #cv2_imshow(images3)
  cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid), img)
  pid2 = pid+'2'
  cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid2), img2)
  pid3 = pid+'3'
  cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid3), img3)
  pid4 = pid+'4'
  cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid4), img4)
  pid5 = pid+'5'
  cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid5), images5)

In [ ]:
print(os.path.isfile(os.path.join(SEGMENTATION_RESULT_TEST,("person1_virus_65"+".png"))))

True


In [ ]:
def gammaCorrection(img_original, gamma):
    ## [changing-contrast-brightness-gamma-correction]
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)

    res = cv2.LUT(img_original, lookUpTable)
    ## [changing-contrast-brightness-gamma-correction]

    #img_gamma_corrected = cv2.hconcat([img_original, res])
    return res

def adjust_gamma(image, gamma=1.0):
   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

   return cv2.LUT(image, table)

def segment_image(pid, img, save_to):
    img = cv2.resize(img, (512, 512))
    segm_ret = segmentation_model.predict(image_to_train(img), \
                                          verbose=0)

    img = cv2.bitwise_and(img, img, mask=train_to_image(segm_ret))
    pid += '3'
    #cv2.imwrite(os.path.join(save_to, "%s.png" % pid), img)
    return img
types = ["*.png","*.jpeg","*.jpg"]

for i in range(0,3):
  for filename in tqdm(glob(os.path.join(RSNA_TEST_DIR, types[i]))):
      pid, fileext = os.path.splitext(os.path.basename(filename))
      #if (pid!="post-intubuation-pneumomediastium-and-pneumothorax-background-covid-19-pneumonia-day6-1"):
      #  continue
      if(os.path.isfile(os.path.join(SEGMENTATION_RESULT_TEST,(pid+".png")))):
        print("skipped "+pid)
        continue
      img = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
      width, height, nChannels, nImages = (512, 512, 0, 2)
      images0 = np.zeros((width, height))
      images0[:, :] = cv2.resize(img, (width, height))
      images1 = np.zeros((width, height))
      images2 = np.zeros((width, height))
      images3 = np.zeros((width, height))
      images4 = np.zeros((width, height))
      images5 = np.zeros((width, height))
      #img = cv2.equalizeHist(img)
      clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,32))
      img2 = clahe.apply(img)
      #clahe = cv2.createCLAHE(clipLimit=0.8, tileGridSize=(8,32))
      #img3 = adjust_gamma(img,gamma=1.5)#clahe.apply(img)
      #img3 = adjust_gamma(img3,gamma=1.5)
      img3 = cv2.equalizeHist(img)
      img3 = gammaCorrection(img3,0.4)
      clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,32))
      img3 = clahe.apply(img3)#cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY))
      clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8,32))
      img4 = clahe.apply(img)
      img = segment_image(pid, img, SEGMENTATION_RESULT_TEST)
      img2 = segment_image(pid, img2, SEGMENTATION_RESULT_TEST)
      img3 = segment_image(pid, img3, SEGMENTATION_RESULT_TEST)
      img4 = segment_image(pid, img4, SEGMENTATION_RESULT_TEST)
      images1[:, :] = cv2.resize(img, (width, height))
      images2[:, :] = cv2.resize(img2, (width, height))
      images3[:, :] = cv2.resize(img3, (width, height))
      images4[:, :] = cv2.resize(img4, (width, height))
      for i in range(0,width):
        for j in range(0,height):
          if (images1[i,j]!=0 or images2[i,j]!=0 or images3[i,j]!=0 or images4[i,j]!=0):
            images5[i,j] = images0[i,j]
          #if (images1[i,j]!=0 and images2[i,j]!=0 and images3[i,j]!=0 and images4[i,j]!=0):
          #  images5[i,j] = images2[i,j]
          #elif (images2[i,j]==0):
          #  if (images1[i,j]==0):
          #    if (images3[i,j]==0):
          #      images5[i,j] = images4[i,j]
          #    else:
          #      images5[i,j] = images3[i,j]
          #  else:
          #    images5[i,j] = images1[i,j]
          #else:
          #  images5[i,j] = images2[i,j]
      #cv2_imshow(img)
      #cv2_imshow(img2)
      #cv2_imshow(images3)
      #cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid), img)
      #pid2 = pid+'2'
      #cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid2), img2)
      #pid3 = pid+'3'
      #cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid3), img3)
      #pid4 = pid+'4'
      #cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid4), img4)
      #pid5 = pid+'5'
      #cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid5), images5)
      pid5 = pid+'0'
      cv2.imwrite(os.path.join(SEGMENTATION_RESULT_TEST, "%s.png" % pid5), images5)

0it [00:00, ?it/s]
100%|██████████| 2/2 [00:25<00:00, 12.84s/it]
0it [00:00, ?it/s]
